In [ ]:
# Vereiste imports

import cv2, json, os
from pandas.io.json import json_normalize
from pytube import YouTube

In [ ]:
# Vermelding naar de freeze-JSON

file = '/path/to/freeze.json'

# Haal alle Youtube-URLs uit de JSON

with open(file, 'r') as f:
    loaded_json = json.load(f)

json_normalize(loaded_json)
data = json_normalize(loaded_json['annotations'])
datay = json_normalize(loaded_json['content'])
datay = datay.fillna('None')
datay = datay[datay['properties.pipeline.download.resource'].apply(lambda x: 'pol_harvester.youtubedlresource' in x)]
datay = datay[~datay["properties.text"].str.contains('None')]
datay = datay[['properties.id','properties.title','properties.text','properties.url','properties.language.from_text']]

url_list = datay["properties.url"].tolist()
url_list_2 = []

for url in url_list:
    if "youtu" in url:
        url_list_2.append(url)

In [ ]:
# Genereer voor elke video 3 frames

count = 0
for url in url_list_2:
    count += 1
    yt = YouTube(url)
    video = yt.streams.get_by_itag(18).download()
    cap = cv2.VideoCapture(video)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameno=int(length/4)
    cap.set(1,frameno); 
    ret, frame = cap.read() 
    filename = "{0}_{1}.jpg".format(count,frameno)
    cv2.imwrite(filename, frame)
    frameno=int((length/4)*2)
    cap.set(1,frameno); 
    ret, frame = cap.read()
    filename = "{0}_{1}.jpg".format(count,frameno)
    cv2.imwrite(filename, frame)
    frameno=int((length/4)*3)
    cap.set(1,frameno);
    ret, frame = cap.read()
    filename = "{0}_{1}.jpg".format(count,frameno)
    cv2.imwrite(filename, frame)
    os.remove(video)